# Neural Network

In [8]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from mnist_get_images import get_images


In [2]:
mnist_path = './mnist_raw/'

In [11]:
X_train_num, y_train_num, X_test_num, y_test_num = get_images(mnist_path)
# Testing data
print(f'Dimensiones datos de entrenamiento: {X_train_num.shape} \nDimensiones datos de prueba: {X_test_num.shape}')

Dimensiones datos de entrenamiento: (60000, 28, 28) 
Dimensiones datos de prueba: (10000, 28, 28)


## Images to vectors in float32